merge dataset


In [ ]:
import pandas as pd
import glob
import os

# =========================
# 1. PATH DATA
# =========================
FOLDER_ISPU = r"E:/Python-Project/Arkavidia-air-polution/ISPU/*.xlsx"
"
# kalau masih xlsx → ganti *.xlsx

files = glob.glob(FOLDER_ISPU)
print(f"Jumlah file ditemukan: {len(files)}")

# =========================
# 2. KAMUS NORMALISASI KOLOM
# =========================
COLUMN_MAP = {
    # tanggal
    "tanggal": "tanggal",
    "date": "tanggal",
    "waktu": "tanggal",
    "tanggal_pengukuran": "tanggal",

    # polutan
    "pm10": "PM10",
    "pm_10": "PM10",

    "pm25": "PM2.5",
    "pm_25": "PM2.5",

    "so2": "SO2",
    "co": "CO",
    "o3": "O3",
    "no2": "NO2",

    # ispu
    "ispu": "ISPU",
    "nilai_ispu": "ISPU",
}

CORE_COLUMNS = [
    "tanggal", "wilayah",
    "PM10", "PM2.5", "SO2", "CO", "O3", "NO2", "ISPU"
]

df_list = []

# =========================
# 3. LOOP SEMUA FILE
# =========================
for file in files:
    # baca file
    if file.endswith(".csv"):
        df = pd.read_csv(file)
    else:
        df = pd.read_excel(file)

    # lowercase dulu biar gampang mapping
    df.columns = [c.lower().strip() for c in df.columns]

    # rename kolom
    df = df.rename(columns=COLUMN_MAP)

    # =========================
    # 4. TAMBAH KOLOM YANG HILANG
    # =========================
    for col in CORE_COLUMNS:
        if col not in df.columns:
            df[col] = pd.NA

    # =========================
    # 5. WILAYAH (DARI NAMA FILE)
    # =========================
    filename = os.path.basename(file).lower()

    if "jakarta" in filename:
        df["wilayah"] = "DKI Jakarta"
    else:
        df["wilayah"] = "Unknown"

    # =========================
    # 6. FORMAT TANGGAL
    # =========================
    df["tanggal"] = pd.to_datetime(df["tanggal"], errors="coerce")

    # ambil kolom inti saja
    df = df[CORE_COLUMNS]

    df_list.append(df)

# =========================
# 7. CONCAT SEMUA
# =========================
df_ispu_all = pd.concat(df_list, ignore_index=True)

# =========================
# 8. SORT TIME-SERIES
# =========================
df_ispu_all = df_ispu_all.sort_values("tanggal").reset_index(drop=True)

print("SELESAI MERGE ✅")
print(df_ispu_all.info())
df_ispu_all.head()


Jumlah file ditemukan: 0


ValueError: No objects to concatenate